In [3]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import numpy as np
from datetime import datetime
from datetime import date
import statsmodels.api as sm
import geopandas as gpd
from geopandas import GeoDataFrame
from shapely.geometry import Point, Polygon
import matplotlib.pyplot as plt 
fig = plt.gcf() 
fig.set_size_inches(12, 8)
import seaborn as sns
%matplotlib inline
from pathlib import Path, PureWindowsPath

In [4]:
shp_file_path = Path("C:/Users/emman/Box/Spring 2020/Displacement Studio/Shared 228 Sydney Folder/SA2 Shapefiles")
dv_analysis_path =  Path("C:/Users/emman/Box/Spring 2020/Displacement Studio/Shared 228 Sydney Folder/AU Work/MasterTypologyFolder")
ab_path = Path("C:/Users/emman/Box/Spring 2020/Displacement Studio/Shared 228 Sydney Folder/AU Work/MasterTypologyFolder")
sa2_w_dv = pd.read_csv(dv_analysis_path/'dummies_t3_corridor.csv')
sa2_shape16 = gpd.read_file(shp_file_path/'sa2_ucl.dbf').copy()
airbnb_df = pd.read_csv('airbnb_sydney_listings.csv')

FileNotFoundError: [Errno 2] File airbnb_sydney_listings.csv does not exist: 'airbnb_sydney_listings.csv'

Let's do some cleaning...

In [ ]:
list(airbnb_df)

In [ ]:
# drop these columns because they're either empty or probably won't be of any use
airbnb_df.drop(['thumbnail_url','medium_url','picture_url','xl_picture_url','host_thumbnail_url',
                'instant_bookable','require_guest_phone_verification','neighbourhood_group_cleansed',
                'host_acceptance_rate','jurisdiction_names','host_url','host_response_time','host_response_rate',
                'host_is_superhost','host_picture_url','host_total_listings_count','host_verifications',
               'minimum_nights','maximum_nights','minimum_minimum_nights','maximum_minimum_nights',
                'minimum_maximum_nights','maximum_maximum_nights','minimum_nights_avg_ntm',
                'maximum_nights_avg_ntm','calendar_updated','number_of_reviews','number_of_reviews_ltm',
                'first_review','last_review','review_scores_rating','review_scores_accuracy',
                'review_scores_cleanliness','review_scores_checkin','review_scores_communication',
                'review_scores_location','review_scores_value',],axis=1,inplace=True)

In [ ]:
# we need to look at what different kinds of properties exist on airbnb and drop the ones that are not homes, 
# apartments, etc. 
airbnb_df.property_type.value_counts()

In [ ]:
# we propbably don't need castle, other, tent, island, train, boat, or any other of the ridiculous values that are in here. I did this line by line so 
# that if you want to add one in, all you have to do is comment out the housing type you want to keep
airbnb_df = airbnb_df[(airbnb_df.property_type != 'Boutique hotel')] 
airbnb_df = airbnb_df[(airbnb_df.property_type != 'Bungalow')] 
airbnb_df = airbnb_df[(airbnb_df.property_type != 'Bed and breakfast')] 
airbnb_df = airbnb_df[(airbnb_df.property_type != 'Hostel')]
airbnb_df = airbnb_df[(airbnb_df.property_type != 'Other')]  
airbnb_df = airbnb_df[(airbnb_df.property_type != 'Hotel')] 
airbnb_df = airbnb_df[(airbnb_df.property_type != 'Boat')] 
airbnb_df = airbnb_df[(airbnb_df.property_type != 'Camper/RV')] 
airbnb_df = airbnb_df[(airbnb_df.property_type != 'Aparthotel')]
airbnb_df = airbnb_df[(airbnb_df.property_type != 'Farm stay')]
airbnb_df = airbnb_df[(airbnb_df.property_type != 'Tent')]
airbnb_df = airbnb_df[(airbnb_df.property_type != 'Earth house')]
airbnb_df = airbnb_df[(airbnb_df.property_type != 'Chalet')]
airbnb_df = airbnb_df[(airbnb_df.property_type != 'Island')]
airbnb_df = airbnb_df[(airbnb_df.property_type != 'Barn')]
airbnb_df = airbnb_df[(airbnb_df.property_type != 'Resort')]
airbnb_df = airbnb_df[(airbnb_df.property_type != 'Casa particular (Cuba)')]
airbnb_df = airbnb_df[(airbnb_df.property_type != 'Yurt')]
airbnb_df = airbnb_df[(airbnb_df.property_type != 'Train')]
airbnb_df = airbnb_df[(airbnb_df.property_type != 'Treehouse')]
airbnb_df = airbnb_df[(airbnb_df.property_type != 'Castle')]
airbnb_df = airbnb_df[(airbnb_df.property_type != 'Heritage hotel (India)')]
airbnb_df = airbnb_df[(airbnb_df.property_type != 'Campsite')]
airbnb_df = airbnb_df[(airbnb_df.property_type != 'Dome house')]
airbnb_df = airbnb_df[(airbnb_df.property_type != 'Hut')]
airbnb_df = airbnb_df[(airbnb_df.property_type != 'Minsu (Taiwan)')]
airbnb_df = airbnb_df[(airbnb_df.property_type != 'Cave')]
airbnb_df = airbnb_df[(airbnb_df.property_type != 'Tipi')]
airbnb_df = airbnb_df[(airbnb_df.property_type != 'Casa particular')]

In [ ]:
airbnb_df.property_type.value_counts()

In [ ]:
# Clean date values to be actual dates
airbnb_df['host_since']=pd.to_datetime(airbnb_df['host_since'])
airbnb_df['last_scraped']=pd.to_datetime(airbnb_df['last_scraped'])

# pull individual year from host_since variable
airbnb_df['hs_year'] = pd.DatetimeIndex(airbnb_df['host_since']).year
airbnb_df['hs_year'].dropna(inplace=True)
airbnb_df['hs_year'].astype(int)

# fill in nulls with 0 or f
airbnb_df.bathrooms.fillna(0,inplace=True)

In [ ]:
# Create a shapely.geometry.Point for each listing
geometry = [Point(xy) for xy in zip(airbnb_df['longitude'], airbnb_df['latitude'])]
crs = {'init' :'epsg:4326'}

# Convert to GeoDataFrame, where each tweet's geometry is assigned to the lat/lon coords
airbnb_gdf = gpd.GeoDataFrame(airbnb_df, crs=crs, geometry=geometry)

In [ ]:
airbnb_df.shape

In [ ]:
# let's isolate the sa2s that make up our case study areas of Redfern/Waterloo, Merrickville and Maroubra
red_wat_gdf = sa2_shape16[(sa2_shape16.SA2_MAIN16=='117031335')
                          |(sa2_shape16.SA2_MAIN16=='117031338')]

sa2_shape16[(sa2_shape16.SA2_MAIN16=='117031335')
                          |(sa2_shape16.SA2_MAIN16=='117031338')]

# now let's create a simple rendition of the df so that we can combine its SA2 boundaries
red_wat_simple = red_wat_gdf[['SA2_MAIN16','geometry', "SA4_NAME16"]].reset_index()

# "dissolve" or "aggregates" all the polygons that share the same value for the variable we call
red_wat_sgdf=red_wat_simple.dissolve(by="SA4_NAME16")
red_wat_sgdf.plot()

In [ ]:
# let's isolate the sa2s that make up our case study areas of Marrickville
marr_gdf = sa2_shape16[(sa2_shape16.SA2_MAIN16=='117021326')
                      |(sa2_shape16.SA2_MAIN16=='117021327')
                      |(sa2_shape16.SA2_MAIN16=='117021328')
                      |(sa2_shape16.SA2_MAIN16=='117021328')]

# now let's create a simple rendition of the df so that we can combine its SA2 boundaries
marr_simple = marr_gdf[['SA2_MAIN16','geometry', "SA4_NAME16"]].reset_index()

# "dissolve" or "aggregates" all the polygons that share the same value for the variable we call
marr_sgdf=marr_simple.dissolve(by="SA4_NAME16")
marr_sgdf.plot()

In [ ]:
sa2_w_dv['SA2_MAIN16']=sa2_w_dv['SA2_MAIN16'].astype(int)
sa2_w_dv.drop('Unnamed: 0',axis=1,inplace=True)
sa2_shape16['SA2_MAIN16']=sa2_shape16['SA2_MAIN16'].astype(int)

In [ ]:
sa2_shape16 = pd.merge(sa2_shape16,sa2_w_dv,on='SA2_MAIN16',how='left')

In [ ]:
sa2_shape16 = gpd.GeoDataFrame(sa2_shape16)

In [ ]:
# let's isolate the sa2s that make up our case study areas of Redfern/Waterloo, Merrickville and Maroubra
t3_corr_gdf = sa2_shape16[(sa2_shape16.t3_corr_dv==1)]

# now let's create a simple rendition of the df so that we can combine its SA2 boundaries
t3_corr_simple = t3_corr_gdf[['SA2_MAIN16','geometry', "SA4_NAME16"]].reset_index()

# "dissolve" or "aggregates" all the polygons that share the same value for the variable we call
t3_corr_sgdf=t3_corr_simple.dissolve(by="SA4_NAME16")
t3_corr_sgdf.plot()

# Calculate Density per SA2

In [ ]:
# assign the CRS to be World Geodetic System 
sa2_shape16.crs = {'init':'epsg:4326'}

In [ ]:
# perform a spatial join on the UCL SA2s to find density of AirBnB listings per SA2
list_density = gpd.sjoin(airbnb_gdf,sa2_shape16,how='right',op='within')

In [ ]:
# pull the info on the new geodataframe
list_density.info()

In [ ]:
# create a group by statement that creates a column of the counts of listings per SA2, or in other words, count the number of geographical points that 
# fall within the boundaries of an SA2 
list_sa2 = pd.DataFrame({'list_per_SA2' : list_density.groupby('SA2_MAIN16')['SA2_MAIN16'].count()}).reset_index()

In [ ]:
# merge the count of listings by SA2 and convert it into a geodataframe so that we can create a choropleth map 
list_sa2 = pd.merge(list_sa2,sa2_shape16,on='SA2_MAIN16',how='left')
list_sa2 = gpd.GeoDataFrame(list_sa2)

In [ ]:
# create a new variable that finds the number of listings per square kilometer 
list_sa2['listing_density'] = list_sa2.list_per_SA2/list_sa2.AREASQKM16

In [ ]:
# plot geodf again with closeups 
figure, ax = plt.subplots(figsize=(14,10))
base= list_sa2.plot(column='list_per_SA2', 
                        scheme = 'fisherjenks',
                        legend=True,  
                        ax=ax,
                        cmap='Reds')

red_wat_sgdf.plot(ax=base, facecolor='none', edgecolor = "Purple", linewidth = 1.5)
marr_sgdf.plot(ax=base, facecolor='none', edgecolor = "Black", linewidth = 1.5)

## Calculating the number of days listings are unavailable. 

In [ ]:
#Grouping by host IDs how many days out of the year, on average, are their listings available?
h_n_list_av = pd.DataFrame(airbnb_df.groupby('property_type').agg({'availability_365':['mean','count']}))
h_n_list_av.reset_index(inplace=True)
h_n_list_av.columns = ['_'.join(col).strip('_') for col in h_n_list_av.columns]
h_n_list_av.head()
# calculate the percentage of the calendar year that these are not available for long term rentals
h_n_list_av['pct_year'] =  round(h_n_list_av['availability_365_mean']/365,3)
h_n_list_av.sort_values(by='availability_365_mean',ascending= False)

In [ ]:
# create a crosstab comparing the average number of days listings are available for Redfern/Waterloo
pd.crosstab(list_density.property_type, list_density.red_wat_dv, values = list_density.availability_365, aggfunc=['mean','count'])

In [ ]:
# create a crosstab comparing the average number of days listings are available for Marrickville
pd.crosstab(list_density.property_type, list_density.marrick_dv, values = list_density.availability_365, aggfunc=['mean','count'])

In [ ]:
# create a crosstab comparing the average number of days listings are available for Sydenham/Bankstown Corridor
pd.crosstab(list_density.property_type, list_density.t3_corr_dv, values = list_density.availability_365, aggfunc=['mean','count'])

## Analyzing the patterns of growth for different AirBnB property types

In [ ]:
airbnb_df['count'] = 1 
prop_year = pd.pivot_table(airbnb_df,index='property_type',columns='hs_year',values='count',aggfunc=np.sum,fill_value=0)
prop_year.reset_index(inplace=True)
prop_year.columns=(['property_type','2009','2010','2011','2012','2013',
                    '2014','2015','2016','2017','2018','2019'])
prop_year_melt = pd.melt(prop_year, 
        id_vars='property_type',
        value_vars = ['2009','2010','2011','2012','2013','2014','2015','2016','2017','2018','2019'],
        value_name='prop_count')
prop_year_melt

In [ ]:
a4_dims = (11.7, 8.27)
fig, ax = plt.subplots(figsize=a4_dims)
ax.set_title('Number of AirBnB Hosts Joining by Property Type, 2009-2019', fontdict={'fontsize':20})
g = sns.lineplot(x='variable', y='prop_count', data=prop_year_melt, hue='property_type',ax=ax)

plt.xlabel('Years')
plt.ylabel('Property Types')

ax.annotate('Source: Inside AirBnB 2019 Scrape Sydney, AU',
            xy=(0.1, .10),  
            xycoords='figure fraction', 
            horizontalalignment='left', 
            verticalalignment='top', 
            fontsize=12, 
            color='#555555')
plt.savefig('hosts_overtime_HnA.png')

In [ ]:
prop_year_melt=prop_year_melt[prop_year_melt.property_type!='Apartment']
prop_year_melt=prop_year_melt[prop_year_melt.property_type!='House']

a4_dims = (11.7, 8.27)
fig, ax = plt.subplots(figsize=a4_dims)
ax.set_title('Number of AirBnB Hosts Joining by Property Type, 2009-2019', fontdict={'fontsize':20})
g = sns.lineplot(x='variable', y='prop_count', data=prop_year_melt, hue='property_type',ax=ax)
plt.xlabel('Years')
plt.ylabel('Property Types')
ax.annotate('Source: Inside AirBnB 2019 Scrape Sydney, AU',
            xy=(0.1, .10),  
            xycoords='figure fraction', 
            horizontalalignment='left', 
            verticalalignment='top', 
            fontsize=12, 
            color='#555555')
plt.savefig('hosts_overtime.png')

## Analyzing trends for number of hosts that have more than one listing in AirBnB

I think it's important to separate out the groups that have more than one AirBnB listing because of our stressing of how financialization of housing affects the market. In our literature review, we found that about 1/5 of Australians own more than one home. If that's the case, let's see how many times there are more than three, and then how the distribution of them look in a scatter plot

### Question 1: Of the hosts that have >= 2 properties, how are they distributed spatially?

Methodology: 

1. Isolate host_ids where hosts own more than 3 properties
1. Let's work on a heatmap where the points on a map show where the distribution of properties that hosts have >= 2 listings are

In [ ]:
from scipy import ndimage
import matplotlib.pylab as pylab
pylab.rcParams['figure.figsize'] = 8, 6

In [ ]:
#create a dataframe out of a groupby that groups host IDs and counts the listing IDs in associated to them
test_df = pd.DataFrame({'cnt_list_host' : airbnb_df.groupby('host_id')['id'].count()}).reset_index()
# create a new geodataframe that only has listing id, host id, and geometry
test_gdf2 = airbnb_gdf[['id','host_id','geometry']]
# test merge 
test_merge = pd.merge(test_df,test_gdf2,on='host_id',how='left')
# isolate new dataframe to only have listings where the count of listings per host are >=2
#test_merge = test_merge[test_merge.cnt_list_host>=2]
# create a new geodataframe
tm_gdf = gpd.GeoDataFrame(test_merge, crs=crs)
#perform a geospatial join on the new test dataframe so that we can perform this at the sa2 level
merge_gdf = gpd.sjoin(tm_gdf,sa2_shape16,how='right',op='within')

In [ ]:
merge_gdf.dropna(inplace=True)
merge_gdf= merge_gdf.reset_index()

In [ ]:
merge_gdf.info()

In [ ]:
merge_gdf.cnt_list_host.value_counts()

In [ ]:
tm_gdf.head()

In [ ]:
figure, ax = plt.subplots(figsize=(14,10))
base = sa2_shape16.plot(ax=ax)
tm_gdf.plot(column="cnt_list_host", 
              scheme = "fisher_jenks", 
              k = 10, 
              legend=True, 
              ax=base, 
              cmap="Reds",
             markersize=.5)

t3_corr_sgdf.plot(ax=base, facecolor='none', edgecolor = "Black", linewidth = 1.5)
red_wat_sgdf.plot(ax=base, facecolor='none', edgecolor = "Purple", linewidth = 1.5)

lims=plt.axis("equal")
ax.set_axis_off()

ax.set_title('Listings where Host Has >=2 Listings', fontdict={'fontsize':25})

ax.annotate('Source: Inside AirBnB, 2019; Australia ABS Census Shapes, 2016',
            xy=(0.1, .08),  
            xycoords='figure fraction', 
            horizontalalignment='left', 
            verticalalignment='top', 
            fontsize=12, 
            color='#555555')

plt.show()